In [1]:
import itertools
from tqdm.notebook import tqdm
import numpy as np
np.random.seed(42)

In [2]:
class TSPGeneticAlgorithm:
  def __init__(self, n, population_size, pc, pm):
    self.n = n
    self.size = population_size
    self.pc = pc
    self.pm = pm
    self.nodes = np.arange(self.n)
    self.graph = [[0 if i == j else np.random.randint(20)+1 for i in range(self.n)] for j in range(self.n)]
    self.__init_population()

  def __fitness(self, x):
    fitness = 0
    for i in range(self.n):
      j = (i + 1) % self.n
      fitness += self.graph[x[i]][x[j]]
    return fitness

  def __init_population(self):
    self.population = [list(np.random.permutation(self.n)) for _ in range(self.size)]
    self.global_best = self.population[0]

  def __selection(self):
    fitness_values = [200 - self.__fitness(p) for p in self.population]
    fitness_prefix = [fitness_values[0]]
    for i in range(1, self.size):
      fitness_prefix.append(fitness_prefix[i-1] + fitness_values[i])
    selected_population = []
    for _ in range(self.size):
      rng = np.random.randint(fitness_prefix[-1])
      for i in range(self.size):
        if rng < fitness_prefix[i]:
          selected_population.append(self.population[i])
          break
    return selected_population

  def __cross(self, x, y):
    n = len(x)
    cut_1 = np.random.randint(n)
    cut_2 = np.random.randint(n)
    if cut_2 < cut_1: cut_1, cut_2 = cut_2, cut_1
    new_x = [None for _ in range(n)]
    new_y = [None for _ in range(n)]
    for i in range(cut_1, cut_2 + 1):
      new_x[i] = y[i]
      new_y[i] = x[i]
    for i in range(cut_1):
      if x[i] not in new_x: new_x[i] = x[i]
      if y[i] not in new_y: new_y[i] = y[i]
    for i in range(cut_2 + 1, n):
      if x[i] not in new_x: new_x[i] = x[i]
      if y[i] not in new_y: new_y[i] = y[i]

    for i in range(n):
      val = x[i]
      while new_x[i] is None:
        j = new_x.index(val)
        if new_y[j] not in new_x:
          new_x[i] = new_y[j]
        else:
          val = new_y[j]
    for i in range(n):
      val = y[i]
      while new_y[i] is None:
        j = new_y.index(val)
        if new_x[j] not in new_y:
          new_y[i] = new_x[j]
        else:
          val = new_x[j]

    return new_x, new_y

  # def __cross(self, x, y):
  #   n = len(x)
  #   cut = np.random.randint(n)
  #   new_x, new_y = x.copy(), y.copy()
  #   for i in range(cut, n):
  #     new_x.remove(y[i])
  #     new_y.remove(x[i])
  #     new_x.append(y[i])
  #     new_y.append(x[i])
  #   return new_x, new_y

    
  def __crossover(self):
    crossed_population = []
    for i in range(0, self.size, 2):
      if np.random.rand() <= self.pc:
        x, y = self.__cross(self.population[i], self.population[i + 1])
      else:
        x, y = self.population[i], self.population[i + 1]
      crossed_population.append(x)
      crossed_population.append(y)
    return crossed_population

  def __mutate(self, x):
    # for i in range(1, len(x)):
    #   if np.random.rand() <= 0.2:
    #     j = np.random.randint(1, len(x))
    #     x[i], x[j] = x[j], x[i]
    # return x
    i = np.random.randint(1, len(x))
    j = np.random.randint(1, len(x))
    x[i], x[j] = x[j], x[i]
    return x

  def __mutation(self):
    mutated_population = []
    for p in self.population:
      if np.random.rand() <= self.pm:
        mutated_population.append(self.__mutate(p))
      else:
        mutated_population.append(p)
    return mutated_population

  def __update_global_best(self):
    best_val = 200
    best_sample = None
    for p in self.population:
      f = self.__fitness(p)
      if f < best_val:
        best_val = f
        best_sample = p
    if self.__fitness(best_sample) < self.__fitness(self.global_best):
      self.global_best = best_sample.copy()

  def __perform_iteration(self):
    self.population = self.__selection()
    self.population = self.__crossover()
    self.population = self.__mutation()
    self.__update_global_best()

  def display_population(self):
    best_val = 200
    best_sample = None
    for p in self.population:
      f = self.__fitness(p)
      print(f'SEQUENCE: {p}, FITNESS: {f}')
      if f < best_val:
        best_val = f
        best_sample = p
    print()
    print(f'BEST SAMPLE: {best_sample}, FITNESS: {best_val}')
    print()

  def get_graph(self):
    return self.graph

  def fit(self, iterations, verbose = True):
    print('INITIAL POPULATION:')
    self.display_population()
    for i in tqdm(range(iterations)):
      self.__perform_iteration()
      if verbose:
        print(f'ITERATION {i+1}:')
        self.display_population()
    print(f'GLOBAL BEST: {self.global_best}, FITNESS: {self.__fitness(self.global_best)}')

  def best_solution(self):
    all_solutions = list(itertools.permutations(self.nodes))
    best_sol = all_solutions[0]
    for solution in all_solutions:
      if self.__fitness(solution) < self.__fitness(best_sol):
        best_sol = solution
    print(f'BEST SOLUTION: {best_sol}, FITNESS: {self.__fitness(best_sol)}')

In [3]:
population = TSPGeneticAlgorithm(8, 20, 0.5, 0.1)

In [4]:
population.fit(10000, verbose = False)

INITIAL POPULATION:
SEQUENCE: [0, 2, 6, 3, 7, 1, 4, 5], FITNESS: 80
SEQUENCE: [4, 2, 1, 0, 6, 7, 3, 5], FITNESS: 65
SEQUENCE: [2, 0, 5, 3, 6, 1, 7, 4], FITNESS: 69
SEQUENCE: [1, 2, 5, 0, 4, 6, 7, 3], FITNESS: 87
SEQUENCE: [2, 1, 7, 3, 5, 6, 0, 4], FITNESS: 73
SEQUENCE: [1, 3, 4, 0, 5, 2, 6, 7], FITNESS: 74
SEQUENCE: [3, 7, 5, 6, 1, 4, 2, 0], FITNESS: 72
SEQUENCE: [2, 4, 5, 1, 7, 0, 6, 3], FITNESS: 92
SEQUENCE: [3, 0, 1, 2, 4, 5, 7, 6], FITNESS: 67
SEQUENCE: [3, 6, 1, 5, 4, 7, 0, 2], FITNESS: 94
SEQUENCE: [5, 3, 6, 7, 2, 1, 4, 0], FITNESS: 53
SEQUENCE: [0, 6, 5, 2, 4, 3, 7, 1], FITNESS: 76
SEQUENCE: [2, 7, 1, 5, 4, 3, 0, 6], FITNESS: 88
SEQUENCE: [5, 1, 3, 0, 2, 4, 7, 6], FITNESS: 95
SEQUENCE: [5, 0, 4, 1, 3, 7, 2, 6], FITNESS: 89
SEQUENCE: [6, 2, 1, 7, 3, 4, 0, 5], FITNESS: 78
SEQUENCE: [5, 4, 6, 7, 0, 3, 1, 2], FITNESS: 113
SEQUENCE: [1, 5, 4, 3, 6, 2, 7, 0], FITNESS: 102
SEQUENCE: [4, 3, 1, 0, 7, 5, 6, 2], FITNESS: 106
SEQUENCE: [5, 7, 6, 4, 3, 1, 2, 0], FITNESS: 74

BEST SAMPLE: [5,

  0%|          | 0/10000 [00:00<?, ?it/s]

GLOBAL BEST: [5, 2, 1, 6, 3, 7, 4, 0], FITNESS: 38


In [5]:
population.best_solution()

BEST SOLUTION: (0, 5, 2, 1, 6, 3, 7, 4), FITNESS: 38
